# Problem Set X: Practicing GPU-Accelerated Python

*LSST DSFP Session 21*

*Instructor: Lehman Garrison (https://github.com/lgarrison)*

Let's revisit some of the problems from Lecture and Problem Set IX and see if we can solve them on the GPU!

**Goals**

In this problem set, you'll:
- convert NumPy code to CuPy
- time the results
- reason about when GPU code will be faster than CPU code
- write a Numba CUDA kernel

## $\pi$ with CuPy

In Lecture IX, we computed the value of $\pi$ by generating $N$ random points and measuring number $N_{\rm circle}$ fell in a circle incribed in a square.  The formula was:

$$
\pi = \frac{4N_{\rm circle}}{N}.
$$

We implemented this NumPy version:

In [1]:
import numpy as np

def numpy_compute_pi(N):
    rng = np.random.default_rng(123)
    pos = rng.random((2, N))
    N_circle = np.sum(np.sum(pos**2, axis=0) < 1)
    return 4 * N_circle / N

%timeit numpy_compute_pi(1_000_000)

18.5 ms ± 64.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Complete the following code to turn this function into a CuPy function that runs on the GPU:

In [2]:
import cupy as cp
stream = cp.cuda.get_current_stream()

def cupy_compute_pi(N):
    # YOUR CODE HERE
    rng = cp.random.default_rng(123)
    pos = rng.random((2, N))
    N_circle = cp.sum(cp.sum(pos**2, axis=0) < 1)
    return 4 * N_circle / N

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <

And time your GPU code. How does it compare to the NumPy version?

In [3]:
%%timeit
cupy_compute_pi(1_000_000)
stream.synchronize()

1.7 ms ± 219 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


It should be ~10x faster. Monte Carlo algorithms are a good example of methods that work well on the GPU because they are highly parallel and all the "data" generation (the random numbers) can happen on the GPU. Only the final result (the measured $\pi$ value) has to be communicated back to the CPU.

This method does still suffer from the same drawback we discussed in Lecture IX, which is using unnecessary temporary memory. In the bonus section, you'll have a chance to implement a smarter method.

## CuPy in the Sky

In Lecture IX, we counted the number of points that fell within angle `theta` of a given `(RA_0, DEC_0)` point in on-sky coordinates. The NumPy version, borrowed from Astropy, looked like:

In [4]:
def numpy_count_sep(RA_0, DEC_0, lon, lat, theta):
    """Based on Astropy's function for computing the angular separation
    between two points on the sphere.

    License: BSD-3-Clause
    """

    sdlon = np.sin(lon - RA_0)
    cdlon = np.cos(lon - RA_0)
    slat1 = np.sin(DEC_0)
    slat2 = np.sin(lat)
    clat1 = np.cos(DEC_0)
    clat2 = np.cos(lat)

    num1 = clat2 * sdlon
    num2 = clat1 * slat2 - slat1 * clat2 * cdlon
    denominator = slat1 * slat2 + clat1 * clat2 * cdlon

    sep = np.arctan2(np.hypot(num1, num2), denominator)

    return np.sum(sep < theta)

Here's how we timed that function:

In [5]:
from astropy.table import Table
rng = np.random.default_rng(123)

N = 10**7
catalog = Table(
    {
        "RA": np.radians(rng.uniform(0, 180, N)),
        "DEC": np.radians(rng.uniform(-90, 90, N)),
    }
)

RA_0 = catalog["RA"][0]
DEC_0 = catalog["DEC"][0]
theta = np.radians(10.0)

%timeit numpy_count_sep(RA_0, DEC_0, catalog['RA'], catalog['DEC'], theta)

1.1 s ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Time this function on the GPU. Let's assume the data is originating on the CPU (as it would in a real application), so you should include the transfer time to and from the GPU.

Here's a few prompts to help you think through this:

1. Do you need to rewrite the function to use CuPy, or can you use it as-is?
1. How can you check that the computation is actually happening on the GPU?
1. What function do you use to send data to the GPU? What function do you use to fetch the data back to the CPU?

In [6]:
%%timeit

# YOUR CODE HERE

RA_gpu = cp.asarray(catalog['RA'])
DEC_gpu = cp.asarray(catalog['DEC'])

RA_0 = RA_gpu[0]
DEC_0 = DEC_gpu[0]
theta = cp.radians(10.0)

res = numpy_count_sep(RA_0, DEC_0, RA_gpu, DEC_gpu, theta)
res.get()

stream.synchronize()

46.5 ms ± 209 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It should be ~25x faster.

## CuPy 2PCF

In Problem Set IX, we went through 3 implementations of a two-point correlation function:

1. a slow version with Python `for` loops,
1. a faster version with NumPy array broadcasting,
1. an even faster version with Numba `for` loops.

CuPy doesn't help us with `for` loops (at least via the NumPy-like interface we've been using), so let's try adapting the NumPy array version to CuPy.

Here's what that version looked like (we'll use fake data points this time):

In [7]:
L = 1.
rng = np.random.default_rng(123)

def numpy_compute_2pcf(pos, R_max, N_bin):
    N = len(pos)

    delta = pos[:, None] - pos
    delta -= L * np.rint(delta / L)  # periodic wrap

    d_ij = np.sqrt(np.sum(delta ** 2, axis=-1))

    hist_edges = np.linspace(0, R_max, N_bin + 1)
    sep_hist = np.histogram(d_ij.flat, bins=hist_edges)[0]

    sep_hist[0] -= N  # remove self-pairs

    RR = N * (N - 1) / L ** 3 * 4/3 * np.pi * (hist_edges[1:]**3 - hist_edges[:-1]**3)

    xi = sep_hist / RR - 1

    return xi


Let's time the NumPy version for 2 values of `N`: `50` and `1000`.

In [8]:
%%timeit pos = rng.uniform(0, L, (50, 3))
numpy_compute_2pcf(pos, 0.1, 100)

161 μs ± 1.72 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%%timeit pos = rng.uniform(0, L, (1_000, 3))
numpy_compute_2pcf(pos, 0.1, 100)

64.6 ms ± 121 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now make a CuPy version of this code. Can you call the `numpy_compute_2pcf()` function unmodified with CuPy arrays, as we did in the Monte-Carlo problem above?

In [10]:
def cupy_compute_2pcf(pos, R_max, N_bin):
    # YOUR CODE HERE
    
    N = len(pos)

    delta = pos[:, None] - pos
    delta -= L * np.rint(delta / L)  # periodic wrap

    d_ij = np.sqrt(np.sum(delta ** 2, axis=-1))

    hist_edges = cp.linspace(0, R_max, N_bin + 1)
    sep_hist = cp.histogram(d_ij.reshape(-1), bins=hist_edges)[0]

    sep_hist[0] -= N  # remove self-pairs

    RR = N * (N - 1) / L ** 3 * 4/3 * np.pi * (hist_edges[1:]**3 - hist_edges[:-1]**3)

    xi = sep_hist / RR - 1

    return xi


In [11]:
%%timeit pos = rng.uniform(0, L, (50, 3))

# YOUR CODE HERE
pos_g = cp.asarray(pos)

cupy_compute_2pcf(pos_g, 0.1, 100).get()
stream.synchronize()

768 μs ± 2.41 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%timeit pos = rng.uniform(0, L, (1000, 3))

# YOUR CODE HERE
pos_g = cp.asarray(pos)

cupy_compute_2pcf(pos_g, 0.1, 100).get()
stream.synchronize()

10.4 ms ± 10.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Interpret the results

You should find that the NumPy version is faster for `N=50`, and the CuPy version is faster for `N=1000`. Why do you think this is? Let's do a quiz to guide your intuition.

**Quiz**

What is (1) the time complexity of sending the data to the GPU, and (2) the time complexity of calculating the 2PCF? Think back to what you learned in Problem Set IX about computational complexity.

1. $\mathcal{O}(1)$, $\mathcal{O}(N)$
1. $\mathcal{O}(1)$, $\mathcal{O}(N^2)$
1. $\mathcal{O}(N)$, $\mathcal{O}(N)$
1. $\mathcal{O}(N)$, $\mathcal{O}(N^2)$

<details>
<summary>Click here for the answer</summary>

It's 4. $\mathcal{O}(N)$, $\mathcal{O}(N^2)$! This is hinting at the fact that for small $N$, the data transfer time to the GPU dominates the amount of work to compute the 2PCF, so we're better off staying on the CPU. But in the limit of large $N$, the amount of 2PCF work will always exceed the data transfer work, so it becomes worthwhile to send the data.

</details>

## Numba CUDA: Monte Carlo $\pi$

Let's use Numba CUDA to accelerate our Monte Carlo $\pi$ computation. Recall how this worked on the CPU in Lecture IX: Numba let us avoid allocating big arrays of random numbers and instead let us generate them on-the-fly.  We'll do the same thing here, except using CUDA kernels.

Make each GPU thread generate `N_iter` points using a `for` loop, and keep a count of `N_circle` as you go. To add the results across all threads, allocate an array with length equal to the number of threads, and have each thread write its result there. Then sum the array using CuPy after the kernel.

To generate the `x` and `y` points, use this code:

```python
x = xoroshiro128p_uniform_float64(rng_states, thread_id)
y = xoroshiro128p_uniform_float64(rng_states, thread_id)
```

In [13]:
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float64

@cuda.jit
def cuda_compute_pi(rng_states, N_iter, out):
    # YOUR CODE HERE
    
    thread_id = cuda.grid(1)

    N_circle = 0
    for i in range(N_iter):
        x = xoroshiro128p_uniform_float64(rng_states, thread_id)
        y = xoroshiro128p_uniform_float64(rng_states, thread_id)
        if x**2 + y**2 < 1:
            N_circle += 1

    out[thread_id] = N_circle

In [14]:
N_iter = 10000
threads_per_block = 64
blocks = 100

N_sample = threads_per_block * blocks * N_iter
print(f'{N_sample=}')

N_sample=64000000


In [15]:
%%timeit
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=123)
out = cp.zeros(threads_per_block * blocks, dtype=np.int64)

cuda_compute_pi[blocks, threads_per_block](rng_states, 10000, out)
cuda.synchronize()
pi = 4. * out.sum() / N_sample
# print(pi)

/mnt/home/lgarrison/lsst-dsfp/venv/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 100 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


6.3 ms ± 437 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now time the CuPy implementation from above. You should find that the Numba CUDA version is 10x faster!

In [16]:
%%timeit
cupy_compute_pi(N_sample)
stream.synchronize()

58.1 ms ± 441 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Project Idea: Numba CUDA 2PCF

It's should be possible to get a big speedup on the 2PCF calculation using Numba CUDA, just like we got a big speedup using Numba instead of NumPy on the CPU. This is certainly trickier than the exercises we've done so far, and the optimal implementation should use shared memory, which we haven't covered. As such, I haven't coded up an answer myself, but I recommend this as a fun challenge!